In [1]:
import torch
import os
import numpy as np
import pickle

In [416]:
file_path_short = 'data/test_short.pkl'
file_path_medium = 'data/test_medium.pkl'
file_path_long = 'data/test_long.pkl'

with open(file_path_short, 'rb') as file:
    test_short = pickle.load(file)

with open(file_path_medium, 'rb') as file:
    test_medium = pickle.load(file)

with open(file_path_long, 'rb') as file:
    test_long = pickle.load(file)

In [418]:
multiple_type_series_medium = []

multiple_type_series_long = []


#Medium Series
for series in test_medium:
    mean_shift = series['mean_shift'].any()
    variance_shift = series['variance_shift'].any()
    trend_shift = series['trend_shift'].any()
    
    # Count the number of active break types
    break_count = sum([mean_shift, variance_shift, trend_shift])
    
    if break_count == 0 or break_count > 1:
        multiple_type_series_medium.append(series)

#Long Series
for series in test_long:
    mean_shift = series['mean_shift'].any()
    variance_shift = series['variance_shift'].any()
    trend_shift = series['trend_shift'].any()
    
    # Count the number of active break types
    break_count = sum([mean_shift, variance_shift, trend_shift])
    
    if break_count == 0 or break_count > 1:
        multiple_type_series_long.append(series)

In [417]:
def convert_tensor(df_list):
    tensor_list = []  # Create a new list to store tensors
    for df in df_list:
        # Extract 'data' column as a NumPy array
        data = df['data'].to_numpy()
        
        # Extract labels as NumPy arrays and convert to float tensors
        labels = {
            'mean_shift': torch.tensor(df['mean_shift'].to_numpy().astype(np.float32)),
            'variance_shift': torch.tensor(df['variance_shift'].to_numpy().astype(np.float32)),
            'trend_shift': torch.tensor(df['trend_shift'].to_numpy().astype(np.float32)),
            'anomaly': torch.tensor(df['anomaly'].to_numpy().astype(np.float32))
        }
        
        # Convert data to PyTorch tensor
        data_tensor = torch.tensor(data, dtype=torch.float32)
        
        # Reshape data for LSTM input ([batch_size=1, sequence_length])
        data_tensor = data_tensor.unsqueeze(0)
        
        # Move data and labels to GPU if available
        if torch.cuda.is_available():
            data_tensor = data_tensor.cuda()
            labels = {key: label.cuda() for key, label in labels.items()}
        
        # Append both data and labels as a dictionary
        tensor_list.append({'data': data_tensor, 'labels': labels})
    
    return tensor_list


In [419]:
test_short = convert_tensor(test_short)
test_medium = convert_tensor(test_medium)
test_long = convert_tensor(test_long)
multiple_type_series_medium = convert_tensor(multiple_type_series_medium)
multiple_type_series_long = convert_tensor(multiple_type_series_long)

In [420]:
torch.cuda.is_available()

True

In [455]:
model_path = f"/cemretez/cemretez/artifacts/best_model_w9xp90xv:v0/model.h5"
model = torch.load(model_path)

# Set to evaluation mode
model.eval()

/tmp/ipykernel_27/1388319753.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)


LSTMModel(
  (lstm): LSTM(1, 256, num_layers=2, batch_first=True, bidirectional=True)
  (linear): Sequential(
    (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(512, 4, kernel_size=(1,), stride=(1,))
  )
)

In [456]:
def compare_break_locations(model, dataset, smoothing_window=5, merge_gap=3):
    data = dataset['data']
    true_labels = dataset['labels']
    output = model(data)
    predicted = (output > 0.0).squeeze(0)

    # Initialize results dictionary
    comparison = {}

    # Helper function: Apply smoothing
    def smooth_predictions(predictions, window_size):
        smoothed = predictions.clone()
        for i in range(len(predictions) - window_size + 1):
            window = predictions[i:i + window_size]
            smoothed[i:i + window_size] = window.sum() > (window_size // 2)
        return smoothed

    # Helper function: Merge breaks
    def merge_close_breaks(predictions, max_gap):
        merged = predictions.clone()
        for i in range(1, len(predictions) - max_gap):
            if predictions[i] == 0 and predictions[i - 1] == 1 and predictions[i + 1] == 1:
                merged[i] = 1  # Merge gap
        return merged

    # Helper function: Extract starting indices of breaks
    def get_start_indices(labels):
        starts = []
        in_block = False

        for i, val in enumerate(labels):
            if val and not in_block:  # Start of a new block
                starts.append(i)
                in_block = True
            elif not val and in_block:  # End of block
                in_block = False

        return starts

    # Helper function: Identify if anomalies are point or collective
    def is_point_anomaly(labels):
        """
        Determine if all anomalies in the given labels are single points.
        """
        contiguous_lengths = []
        in_anomaly = False

        for i, val in enumerate(labels):
            if val and not in_anomaly:
                in_anomaly = True
                start_idx = i
            elif not val and in_anomaly:
                in_anomaly = False
                contiguous_lengths.append(i - start_idx)
        if in_anomaly:  # Handle end of series case
            contiguous_lengths.append(len(labels) - start_idx)

        # Check if all anomalies are single points
        return all(length == 1 for length in contiguous_lengths)

    # Process each class
    for i, class_name in enumerate(true_labels.keys()):
        pred = predicted[i]  # Predictions for the current class
        true = true_labels[class_name]  # True labels for the current class

        if class_name == "anomaly":
            # Determine if true anomalies are point anomalies or collective
            if is_point_anomaly(true):
                # For point anomalies, directly extract indices of all `1`s
                predicted_start_indices = torch.nonzero(pred, as_tuple=True)[0].tolist()
                true_start_indices = torch.nonzero(true, as_tuple=True)[0].tolist()
            else:
                # For collective anomalies, apply smoothing and merging
                pred = smooth_predictions(pred, smoothing_window)
                pred = merge_close_breaks(pred, merge_gap)
                true = smooth_predictions(true.clone(), smoothing_window)
                true = merge_close_breaks(true, merge_gap)

                # Extract starting indices of blocks
                predicted_start_indices = get_start_indices(pred.tolist())
                true_start_indices = get_start_indices(true.tolist())
        else:
            # For other break types, apply smoothing and merging
            pred = smooth_predictions(pred, smoothing_window)
            pred = merge_close_breaks(pred, merge_gap)

            # Extract starting indices of breaks
            predicted_start_indices = get_start_indices(pred.tolist())
            true_start_indices = get_start_indices(true.tolist())

        # Store results
        comparison[class_name] = {
            "predicted_start_indices": predicted_start_indices,
            "true_start_indices": true_start_indices
        }

    return comparison


In [457]:
def process_output(output, tolerance=0):
    results = {}
    
    for break_type, indices in output.items():
        predicted = indices['predicted_start_indices']
        true = indices['true_start_indices']
        
        # Detection: Was the break detected?
        detection = len(predicted) > 0 and len(true) > 0
        
        # Calculate locational accuracy
        locational_matches = []
        unmatched_predicted = []
        unmatched_true = []
        
        for true_start in true:
            matched = False
            for pred_start in predicted:
                if abs(pred_start - true_start) <= tolerance:
                    locational_matches.append((true_start, pred_start))
                    matched = True
                    break
            if not matched:
                unmatched_true.append(true_start)
        
        # Any predicted breaks not matched to true breaks
        unmatched_predicted = [
            pred_start for pred_start in predicted
            if not any(abs(pred_start - true_start) <= tolerance for true_start in true)
        ]
        
        results[break_type] = {
            'detection': detection,
            'locational_matches': locational_matches,
            'unmatched_true': unmatched_true,
            'unmatched_predicted': unmatched_predicted,
        }
    
    return results

In [458]:
def analyze_series_by_break_type(results):
    # Initialize metrics dictionary
    metrics_by_type = {}
    locational_accuracy_by_type = {}

    # Iterate through results (series-level data)
    for series_results in results:
        for break_type, details in series_results.items():
            if break_type not in metrics_by_type:
                # Initialize metrics for each break type
                metrics_by_type[break_type] = {
                    "true_positives": 0,
                    "false_positives": 0,
                    "true_negatives": 0,
                    "false_negatives": 0,
                    "total_series": 0,
                }
                locational_accuracy_by_type[break_type] = {
                    "series_with_perfect_location_accuracy": 0,
                    "series_with_partial_location_accuracy": 0,
                    "series_with_no_location_accuracy": 0,
                    "total_mae": 0,
                    "total_mse": 0,
                }

            # Extract details
            detection = details["detection"]
            locational_matches = details["locational_matches"]
            unmatched_true = details["unmatched_true"]
            unmatched_predicted = details["unmatched_predicted"]

            # Update total series count for this break type
            metrics_by_type[break_type]["total_series"] += 1

            # Classify series performance for detection
            if not unmatched_true and not unmatched_predicted and not detection:
                # True Negative
                metrics_by_type[break_type]["true_negatives"] += 1
                continue  # Skip location accuracy evaluation
            elif not unmatched_true and unmatched_predicted:
                # False Positive
                metrics_by_type[break_type]["false_positives"] += 1
                continue  # Skip location accuracy evaluation
            elif unmatched_true and not detection:
                # False Negative
                metrics_by_type[break_type]["false_negatives"] += 1
                locational_accuracy_by_type[break_type]["series_with_no_location_accuracy"] += 1
            else:
                # True Positive
                metrics_by_type[break_type]["true_positives"] += 1

                # Classify locational accuracy
                if not unmatched_true and not locational_matches:
                    # Skip series without true shifts
                    continue
                elif len(locational_matches) == len(unmatched_true) + len(locational_matches):
                    # Perfect Location Accuracy
                    locational_accuracy_by_type[break_type]["series_with_perfect_location_accuracy"] += 1
                elif len(locational_matches) > 0:
                    # Partial Location Accuracy
                    locational_accuracy_by_type[break_type]["series_with_partial_location_accuracy"] += 1
                else:
                    # No Location Accuracy
                    locational_accuracy_by_type[break_type]["series_with_no_location_accuracy"] += 1

                # Calculate MAE and MSE for locational matches
                diffs = [abs(true - pred) for true, pred in locational_matches]
                if diffs:
                    locational_accuracy_by_type[break_type]["total_mae"] += sum(diffs)
                    locational_accuracy_by_type[break_type]["total_mse"] += sum(d**2 for d in diffs)

    # Finalize metrics for each break type
    for break_type in metrics_by_type.keys():
        # Calculate detection rates
        tp = metrics_by_type[break_type]["true_positives"]
        fp = metrics_by_type[break_type]["false_positives"]
        fn = metrics_by_type[break_type]["false_negatives"]
        tn = metrics_by_type[break_type]["true_negatives"]
        total = metrics_by_type[break_type]["total_series"]

        accuracy = (tp + tn) / total if total > 0 else 0
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics_by_type[break_type].update({
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score,
        })

        # Calculate average MAE and MSE
        loc_acc = locational_accuracy_by_type[break_type]
        loc_acc["average_mae_per_series"] = loc_acc["total_mae"] / loc_acc["series_with_perfect_location_accuracy"] if loc_acc["series_with_perfect_location_accuracy"] > 0 else None
        loc_acc["average_mse_per_series"] = loc_acc["total_mse"] / loc_acc["series_with_perfect_location_accuracy"] if loc_acc["series_with_perfect_location_accuracy"] > 0 else None

    return metrics_by_type, locational_accuracy_by_type


In [289]:
processed_outputs = []
for series in test_short:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=0))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [290]:
print(f'Performance Analysis of Short Series(tolerance=0): {result_analysis} ')

Performance Analysis of Short Series(tolerance=0): ({'mean_shift': {'true_positives': 384, 'false_positives': 46, 'true_negatives': 1030, 'false_negatives': 32, 'total_series': 1492, 'accuracy': 0.9477211796246648, 'precision': 0.8930232558139535, 'recall': 0.9230769230769231, 'f1_score': 0.9078014184397164}, 'variance_shift': {'true_positives': 351, 'false_positives': 41, 'true_negatives': 1031, 'false_negatives': 69, 'total_series': 1492, 'accuracy': 0.9262734584450402, 'precision': 0.8954081632653061, 'recall': 0.8357142857142857, 'f1_score': 0.8645320197044336}, 'trend_shift': {'true_positives': 208, 'false_positives': 55, 'true_negatives': 1161, 'false_negatives': 68, 'total_series': 1492, 'accuracy': 0.9175603217158177, 'precision': 0.7908745247148289, 'recall': 0.7536231884057971, 'f1_score': 0.7717996289424861}, 'anomaly': {'true_positives': 210, 'false_positives': 36, 'true_negatives': 1191, 'false_negatives': 55, 'total_series': 1492, 'accuracy': 0.9390080428954424, 'precisio

In [291]:
processed_outputs = []
for series in test_short:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=3))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [292]:
print(f'Performance Analysis of Short Series(tolerance=3): {result_analysis} ')

Performance Analysis of Short Series(tolerance=3): ({'mean_shift': {'true_positives': 382, 'false_positives': 48, 'true_negatives': 1030, 'false_negatives': 32, 'total_series': 1492, 'accuracy': 0.9463806970509383, 'precision': 0.8883720930232558, 'recall': 0.9227053140096618, 'f1_score': 0.9052132701421801}, 'variance_shift': {'true_positives': 335, 'false_positives': 57, 'true_negatives': 1031, 'false_negatives': 69, 'total_series': 1492, 'accuracy': 0.9155495978552279, 'precision': 0.8545918367346939, 'recall': 0.8292079207920792, 'f1_score': 0.8417085427135678}, 'trend_shift': {'true_positives': 204, 'false_positives': 59, 'true_negatives': 1161, 'false_negatives': 68, 'total_series': 1492, 'accuracy': 0.9148793565683646, 'precision': 0.7756653992395437, 'recall': 0.75, 'f1_score': 0.7626168224299066}, 'anomaly': {'true_positives': 211, 'false_positives': 35, 'true_negatives': 1191, 'false_negatives': 55, 'total_series': 1492, 'accuracy': 0.9396782841823056, 'precision': 0.85772357

In [293]:
processed_outputs = []
for series in test_short:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=5))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [294]:
print(f'Performance Analysis of Short Series(tolerance=5): {result_analysis} ')

Performance Analysis of Short Series(tolerance=5): ({'mean_shift': {'true_positives': 381, 'false_positives': 49, 'true_negatives': 1030, 'false_negatives': 32, 'total_series': 1492, 'accuracy': 0.9457104557640751, 'precision': 0.8860465116279069, 'recall': 0.9225181598062954, 'f1_score': 0.9039145907473309}, 'variance_shift': {'true_positives': 332, 'false_positives': 60, 'true_negatives': 1031, 'false_negatives': 69, 'total_series': 1492, 'accuracy': 0.9135388739946381, 'precision': 0.8469387755102041, 'recall': 0.827930174563591, 'f1_score': 0.8373266078184111}, 'trend_shift': {'true_positives': 197, 'false_positives': 66, 'true_negatives': 1161, 'false_negatives': 68, 'total_series': 1492, 'accuracy': 0.9101876675603218, 'precision': 0.7490494296577946, 'recall': 0.7433962264150943, 'f1_score': 0.7462121212121211}, 'anomaly': {'true_positives': 212, 'false_positives': 34, 'true_negatives': 1191, 'false_negatives': 55, 'total_series': 1492, 'accuracy': 0.9403485254691689, 'precision

In [295]:
processed_outputs = []
for series in test_medium:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=0))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [296]:
print(f'Performance Analysis of Medium Series(tolerance=0): {result_analysis} ')

Performance Analysis of Medium Series(tolerance=0): ({'mean_shift': {'true_positives': 996, 'false_positives': 19, 'true_negatives': 1638, 'false_negatives': 27, 'total_series': 2680, 'accuracy': 0.9828358208955223, 'precision': 0.9812807881773399, 'recall': 0.9736070381231672, 'f1_score': 0.9774288518155054}, 'variance_shift': {'true_positives': 1044, 'false_positives': 10, 'true_negatives': 1620, 'false_negatives': 6, 'total_series': 2680, 'accuracy': 0.9940298507462687, 'precision': 0.9905123339658444, 'recall': 0.9942857142857143, 'f1_score': 0.9923954372623573}, 'trend_shift': {'true_positives': 1192, 'false_positives': 12, 'true_negatives': 1404, 'false_negatives': 72, 'total_series': 2680, 'accuracy': 0.9686567164179104, 'precision': 0.9900332225913622, 'recall': 0.9430379746835443, 'f1_score': 0.965964343598055}, 'anomaly': {'true_positives': 240, 'false_positives': 14, 'true_negatives': 2416, 'false_negatives': 10, 'total_series': 2680, 'accuracy': 0.991044776119403, 'precisio

In [297]:
processed_outputs = []
for series in test_medium:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=3))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [298]:
print(f'Performance Analysis of Medium Series(tolerance=3): {result_analysis} ')

Performance Analysis of Medium Series(tolerance=3): ({'mean_shift': {'true_positives': 974, 'false_positives': 41, 'true_negatives': 1638, 'false_negatives': 27, 'total_series': 2680, 'accuracy': 0.9746268656716418, 'precision': 0.9596059113300492, 'recall': 0.973026973026973, 'f1_score': 0.9662698412698414}, 'variance_shift': {'true_positives': 1018, 'false_positives': 36, 'true_negatives': 1620, 'false_negatives': 6, 'total_series': 2680, 'accuracy': 0.9843283582089553, 'precision': 0.9658444022770398, 'recall': 0.994140625, 'f1_score': 0.9797882579403273}, 'trend_shift': {'true_positives': 1177, 'false_positives': 27, 'true_negatives': 1404, 'false_negatives': 72, 'total_series': 2680, 'accuracy': 0.9630597014925373, 'precision': 0.9775747508305648, 'recall': 0.9423538831064852, 'f1_score': 0.959641255605381}, 'anomaly': {'true_positives': 238, 'false_positives': 16, 'true_negatives': 2416, 'false_negatives': 10, 'total_series': 2680, 'accuracy': 0.9902985074626866, 'precision': 0.9

In [299]:
processed_outputs = []
for series in test_medium:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=5))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [300]:
print(f'Performance Analysis of Medium Series(tolerance=5): {result_analysis} ')

Performance Analysis of Medium Series(tolerance=5): ({'mean_shift': {'true_positives': 970, 'false_positives': 45, 'true_negatives': 1638, 'false_negatives': 27, 'total_series': 2680, 'accuracy': 0.9731343283582089, 'precision': 0.9556650246305419, 'recall': 0.9729187562688064, 'f1_score': 0.9642147117296223}, 'variance_shift': {'true_positives': 1009, 'false_positives': 45, 'true_negatives': 1620, 'false_negatives': 6, 'total_series': 2680, 'accuracy': 0.9809701492537314, 'precision': 0.9573055028462998, 'recall': 0.994088669950739, 'f1_score': 0.9753504108264862}, 'trend_shift': {'true_positives': 1169, 'false_positives': 35, 'true_negatives': 1404, 'false_negatives': 72, 'total_series': 2680, 'accuracy': 0.9600746268656717, 'precision': 0.9709302325581395, 'recall': 0.9419822723609992, 'f1_score': 0.9562372188139059}, 'anomaly': {'true_positives': 238, 'false_positives': 16, 'true_negatives': 2416, 'false_negatives': 10, 'total_series': 2680, 'accuracy': 0.9902985074626866, 'precisi

In [301]:
processed_outputs = []
for series in test_long:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=0))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [302]:
print(f'Performance Analysis of Long Series(tolerance=0): {result_analysis} ')

Performance Analysis of Long Series(tolerance=0): ({'mean_shift': {'true_positives': 1027, 'false_positives': 1, 'true_negatives': 1651, 'false_negatives': 1, 'total_series': 2680, 'accuracy': 0.9992537313432835, 'precision': 0.9990272373540856, 'recall': 0.9990272373540856, 'f1_score': 0.9990272373540856}, 'variance_shift': {'true_positives': 1052, 'false_positives': 3, 'true_negatives': 1625, 'false_negatives': 0, 'total_series': 2680, 'accuracy': 0.9988805970149254, 'precision': 0.9971563981042654, 'recall': 1.0, 'f1_score': 0.9985761746559088}, 'trend_shift': {'true_positives': 1204, 'false_positives': 2, 'true_negatives': 1415, 'false_negatives': 59, 'total_series': 2680, 'accuracy': 0.9772388059701492, 'precision': 0.9983416252072969, 'recall': 0.953285827395091, 'f1_score': 0.9752936411502633}, 'anomaly': {'true_positives': 232, 'false_positives': 8, 'true_negatives': 2439, 'false_negatives': 1, 'total_series': 2680, 'accuracy': 0.9966417910447761, 'precision': 0.966666666666666

In [303]:
processed_outputs = []
for series in test_long:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=3))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [304]:
print(f'Performance Analysis of Long Series(tolerance=3): {result_analysis} ')

Performance Analysis of Long Series(tolerance=3): ({'mean_shift': {'true_positives': 1018, 'false_positives': 10, 'true_negatives': 1651, 'false_negatives': 1, 'total_series': 2680, 'accuracy': 0.9958955223880597, 'precision': 0.9902723735408561, 'recall': 0.9990186457311089, 'f1_score': 0.9946262823644357}, 'variance_shift': {'true_positives': 1034, 'false_positives': 21, 'true_negatives': 1625, 'false_negatives': 0, 'total_series': 2680, 'accuracy': 0.9921641791044776, 'precision': 0.9800947867298578, 'recall': 1.0, 'f1_score': 0.9899473432264241}, 'trend_shift': {'true_positives': 1203, 'false_positives': 3, 'true_negatives': 1415, 'false_negatives': 59, 'total_series': 2680, 'accuracy': 0.9768656716417911, 'precision': 0.9975124378109452, 'recall': 0.9532488114104596, 'f1_score': 0.9748784440842787}, 'anomaly': {'true_positives': 230, 'false_positives': 10, 'true_negatives': 2439, 'false_negatives': 1, 'total_series': 2680, 'accuracy': 0.9958955223880597, 'precision': 0.95833333333

In [305]:
processed_outputs = []
for series in test_long:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=5))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [306]:
print(f'Performance Analysis of Long Series(tolerance=5): {result_analysis} ')

Performance Analysis of Long Series(tolerance=5): ({'mean_shift': {'true_positives': 1017, 'false_positives': 11, 'true_negatives': 1651, 'false_negatives': 1, 'total_series': 2680, 'accuracy': 0.9955223880597015, 'precision': 0.9892996108949417, 'recall': 0.9990176817288802, 'f1_score': 0.9941348973607038}, 'variance_shift': {'true_positives': 1027, 'false_positives': 28, 'true_negatives': 1625, 'false_negatives': 0, 'total_series': 2680, 'accuracy': 0.9895522388059701, 'precision': 0.9734597156398104, 'recall': 1.0, 'f1_score': 0.9865513928914506}, 'trend_shift': {'true_positives': 1199, 'false_positives': 7, 'true_negatives': 1415, 'false_negatives': 59, 'total_series': 2680, 'accuracy': 0.9753731343283583, 'precision': 0.9941956882255389, 'recall': 0.9531001589825119, 'f1_score': 0.9732142857142857}, 'anomaly': {'true_positives': 229, 'false_positives': 11, 'true_negatives': 2439, 'false_negatives': 1, 'total_series': 2680, 'accuracy': 0.9955223880597015, 'precision': 0.95416666666

In [307]:
processed_outputs = []
for series in multiple_type_series_medium:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=0))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [308]:
print(f'Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=0): {result_analysis} ')

Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=0): ({'mean_shift': {'true_positives': 586, 'false_positives': 8, 'true_negatives': 809, 'false_negatives': 21, 'total_series': 1424, 'accuracy': 0.9796348314606742, 'precision': 0.9865319865319865, 'recall': 0.9654036243822076, 'f1_score': 0.9758534554537885}, 'variance_shift': {'true_positives': 627, 'false_positives': 3, 'true_negatives': 790, 'false_negatives': 4, 'total_series': 1424, 'accuracy': 0.9950842696629213, 'precision': 0.9952380952380953, 'recall': 0.993660855784469, 'f1_score': 0.9944488501189532}, 'trend_shift': {'true_positives': 840, 'false_positives': 12, 'true_negatives': 568, 'false_negatives': 4, 'total_series': 1424, 'accuracy': 0.9887640449438202, 'precision': 0.9859154929577465, 'recall': 0.995260663507109, 'f1_score': 0.9905660377358492}, 'anomaly': {'true_positives': 240, 'false_positives': 6, 'true_negatives': 1168, 'false_negatives': 10, 'total_series': 1424, 'accuracy': 0.9887640449

In [309]:
processed_outputs = []
for series in multiple_type_series_medium:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=3))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [310]:
print(f'Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=3): {result_analysis} ')

Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=3): ({'mean_shift': {'true_positives': 573, 'false_positives': 21, 'true_negatives': 809, 'false_negatives': 21, 'total_series': 1424, 'accuracy': 0.9705056179775281, 'precision': 0.9646464646464646, 'recall': 0.9646464646464646, 'f1_score': 0.9646464646464646}, 'variance_shift': {'true_positives': 612, 'false_positives': 18, 'true_negatives': 790, 'false_negatives': 4, 'total_series': 1424, 'accuracy': 0.9845505617977528, 'precision': 0.9714285714285714, 'recall': 0.9935064935064936, 'f1_score': 0.9823434991974318}, 'trend_shift': {'true_positives': 834, 'false_positives': 18, 'true_negatives': 568, 'false_negatives': 4, 'total_series': 1424, 'accuracy': 0.9845505617977528, 'precision': 0.9788732394366197, 'recall': 0.9952267303102625, 'f1_score': 0.9869822485207101}, 'anomaly': {'true_positives': 238, 'false_positives': 8, 'true_negatives': 1168, 'false_negatives': 10, 'total_series': 1424, 'accuracy': 0.987359

In [311]:
processed_outputs = []
for series in multiple_type_series_medium:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=3))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [312]:
print(f'Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=5): {result_analysis} ')

Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=5): ({'mean_shift': {'true_positives': 573, 'false_positives': 21, 'true_negatives': 809, 'false_negatives': 21, 'total_series': 1424, 'accuracy': 0.9705056179775281, 'precision': 0.9646464646464646, 'recall': 0.9646464646464646, 'f1_score': 0.9646464646464646}, 'variance_shift': {'true_positives': 612, 'false_positives': 18, 'true_negatives': 790, 'false_negatives': 4, 'total_series': 1424, 'accuracy': 0.9845505617977528, 'precision': 0.9714285714285714, 'recall': 0.9935064935064936, 'f1_score': 0.9823434991974318}, 'trend_shift': {'true_positives': 834, 'false_positives': 18, 'true_negatives': 568, 'false_negatives': 4, 'total_series': 1424, 'accuracy': 0.9845505617977528, 'precision': 0.9788732394366197, 'recall': 0.9952267303102625, 'f1_score': 0.9869822485207101}, 'anomaly': {'true_positives': 238, 'false_positives': 8, 'true_negatives': 1168, 'false_negatives': 10, 'total_series': 1424, 'accuracy': 0.987359

In [313]:
processed_outputs = []
for series in multiple_type_series_long:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=0))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [314]:
print(f'Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=0): {result_analysis} ')

Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=0): ({'mean_shift': {'true_positives': 611, 'false_positives': 1, 'true_negatives': 811, 'false_negatives': 1, 'total_series': 1424, 'accuracy': 0.9985955056179775, 'precision': 0.9983660130718954, 'recall': 0.9983660130718954, 'f1_score': 0.9983660130718954}, 'variance_shift': {'true_positives': 632, 'false_positives': 2, 'true_negatives': 790, 'false_negatives': 0, 'total_series': 1424, 'accuracy': 0.9985955056179775, 'precision': 0.9968454258675079, 'recall': 1.0, 'f1_score': 0.9984202211690364}, 'trend_shift': {'true_positives': 844, 'false_positives': 1, 'true_negatives': 579, 'false_negatives': 0, 'total_series': 1424, 'accuracy': 0.9992977528089888, 'precision': 0.9988165680473373, 'recall': 1.0, 'f1_score': 0.9994079336885732}, 'anomaly': {'true_positives': 232, 'false_positives': 3, 'true_negatives': 1188, 'false_negatives': 1, 'total_series': 1424, 'accuracy': 0.9971910112359551, 'precision': 0.98723404

In [315]:
processed_outputs = []
for series in multiple_type_series_long:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=3))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [316]:
print(f'Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=3): {result_analysis} ')

Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=3): ({'mean_shift': {'true_positives': 604, 'false_positives': 8, 'true_negatives': 811, 'false_negatives': 1, 'total_series': 1424, 'accuracy': 0.9936797752808989, 'precision': 0.9869281045751634, 'recall': 0.9983471074380166, 'f1_score': 0.9926047658175843}, 'variance_shift': {'true_positives': 620, 'false_positives': 14, 'true_negatives': 790, 'false_negatives': 0, 'total_series': 1424, 'accuracy': 0.9901685393258427, 'precision': 0.9779179810725552, 'recall': 1.0, 'f1_score': 0.9888357256778308}, 'trend_shift': {'true_positives': 844, 'false_positives': 1, 'true_negatives': 579, 'false_negatives': 0, 'total_series': 1424, 'accuracy': 0.9992977528089888, 'precision': 0.9988165680473373, 'recall': 1.0, 'f1_score': 0.9994079336885732}, 'anomaly': {'true_positives': 230, 'false_positives': 5, 'true_negatives': 1188, 'false_negatives': 1, 'total_series': 1424, 'accuracy': 0.9957865168539326, 'precision': 0.9787234

In [318]:
processed_outputs = []
for series in multiple_type_series_long:
    result = (compare_break_locations(model,series))
    processed_outputs.append(process_output(result, tolerance=5))

result_analysis = analyze_series_by_break_type(processed_outputs)

In [319]:
print(f'Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=5): {result_analysis} ')

Performance Analysis of Medium Series with Multiple Type Shifts(tolerance=5): ({'mean_shift': {'true_positives': 603, 'false_positives': 9, 'true_negatives': 811, 'false_negatives': 1, 'total_series': 1424, 'accuracy': 0.9929775280898876, 'precision': 0.9852941176470589, 'recall': 0.9983443708609272, 'f1_score': 0.9917763157894737}, 'variance_shift': {'true_positives': 616, 'false_positives': 18, 'true_negatives': 790, 'false_negatives': 0, 'total_series': 1424, 'accuracy': 0.9873595505617978, 'precision': 0.9716088328075709, 'recall': 1.0, 'f1_score': 0.9856}, 'trend_shift': {'true_positives': 843, 'false_positives': 2, 'true_negatives': 579, 'false_negatives': 0, 'total_series': 1424, 'accuracy': 0.9985955056179775, 'precision': 0.9976331360946745, 'recall': 1.0, 'f1_score': 0.9988151658767772}, 'anomaly': {'true_positives': 229, 'false_positives': 6, 'true_negatives': 1188, 'false_negatives': 1, 'total_series': 1424, 'accuracy': 0.9950842696629213, 'precision': 0.9744680851063829, '

In [ ]:
###For comparison with Pettitt, Zivot-Andrews and Bartlett###

In [444]:
test_short[1]['labels']

{'mean_shift': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.], device='cuda:0'),
 'variance_shift': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.], device='cuda:0'),
 'trend_shift': tensor([0., 0., 0., 0., 

In [448]:
def filter_series_for_specific_break(dataset, break_type, max_breaks=1):
    """
    Filters the dataset for a specific break type or no breaks at all.
    
    Args:
        dataset: List of series, where each series has 'data' and labels for different break types.
        break_type: The type of break to filter for ('mean_shift', 'variance_shift', 'trend_shift').
        max_breaks: Maximum number of breaks of the specified type allowed (default is 1).
    
    Returns:
        Filtered list of series.
    """
    def count_break_regions(labels):
        """Counts the number of distinct break regions."""
        if isinstance(labels, (list, tuple)):
            labels = [bool(x) for x in labels]  # Ensure labels are boolean
        return sum(labels[i] and (i == 0 or not labels[i - 1]) for i in range(len(labels)))

    filtered = []
    for series in dataset:
        labels = series["labels"]  # Assuming labels are stored under the "labels" key
        
        # Count breaks for the specified break type
        num_breaks = count_break_regions(labels.get(break_type, []))
        
        # Ensure no regions for other break-related types
        break_related_columns = ['mean_shift', 'variance_shift', 'trend_shift']
        break_related_columns.remove(break_type)
        no_other_breaks = all(
            count_break_regions(labels.get(bt, [])) == 0
            for bt in break_related_columns
        )
        
        # Check for no-break cases
        no_breaks = all(
            all(not x for x in labels.get(bt, []))  # Ensure all values are False
            for bt in ['mean_shift', 'variance_shift', 'trend_shift']
        )
        
        # Add to filtered list if conditions are met
        if (num_breaks == max_breaks and no_other_breaks) or no_breaks:
            filtered.append(series)
    
    return filtered


In [450]:
filtered_mean_shift_short = filter_series_for_specific_break(test_short, 'mean_shift', max_breaks=1)
filtered_trend_shift_short = filter_series_for_specific_break(test_short, 'trend_shift', max_breaks=1)

filtered_mean_shift_medium = filter_series_for_specific_break(test_medium, 'mean_shift', max_breaks=1)
filtered_trend_shift_medium = filter_series_for_specific_break(test_medium, 'trend_shift', max_breaks=1)

filtered_mean_shift_long = filter_series_for_specific_break(test_long, 'mean_shift', max_breaks=1)
filtered_trend_shift_long = filter_series_for_specific_break(test_long, 'trend_shift', max_breaks=1)


In [451]:
filtered_mean_shift = filtered_mean_shift_short + filtered_mean_shift_medium + filtered_mean_shift_long
filtered_trend_shift = filtered_trend_shift_short + filtered_trend_shift_medium + filtered_trend_shift_long

In [459]:
for dataset, break_type in zip(
    [filtered_mean_shift],  # Only process mean shift series
    ['mean_shift']  # Corresponding break type
):
    # List to store processed outputs for the current break type
    processed_outputs = []

    # Process each series in the dataset
    for series in dataset:
        # Directly pass the entire series (dataframe) to compare_break_locations
        # This ensures both 'data' and relevant 'labels' are available
        comparison_result = compare_break_locations(model, series)
        
        # Process comparison result with the desired tolerance
        processed_result = process_output(comparison_result, tolerance=0)
        
        # Append processed result to the list
        processed_outputs.append(processed_result)
    
    # Analyze results for the current break type
    result_analysis = analyze_series_by_break_type(processed_outputs)
    
    # Print the analysis for this break type
    print(f"Analysis for series with {break_type}(0):")
    print(result_analysis)

Analysis for series with mean_shift(0):
({'mean_shift': {'true_positives': 730, 'false_positives': 20, 'true_negatives': 1120, 'false_negatives': 17, 'total_series': 1887, 'accuracy': 0.9803921568627451, 'precision': 0.9733333333333334, 'recall': 0.9772423025435074, 'f1_score': 0.9752839011356046}, 'variance_shift': {'true_positives': 0, 'false_positives': 35, 'true_negatives': 1852, 'false_negatives': 0, 'total_series': 1887, 'accuracy': 0.9814520402755696, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'trend_shift': {'true_positives': 0, 'false_positives': 48, 'true_negatives': 1839, 'false_negatives': 0, 'total_series': 1887, 'accuracy': 0.9745627980922098, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'anomaly': {'true_positives': 682, 'false_positives': 39, 'true_negatives': 1100, 'false_negatives': 66, 'total_series': 1887, 'accuracy': 0.9443561208267091, 'precision': 0.9459084604715673, 'recall': 0.9117647058823529, 'f1_score': 0.9285228046289994}}, {'mean_shift': {'series_wit

In [460]:
for dataset, break_type in zip(
    [filtered_mean_shift],  # Only process mean shift series
    ['mean_shift']  # Corresponding break type
):
    # List to store processed outputs for the current break type
    processed_outputs = []

    # Process each series in the dataset
    for series in dataset:
        # Directly pass the entire series (dataframe) to compare_break_locations
        # This ensures both 'data' and relevant 'labels' are available
        comparison_result = compare_break_locations(model, series)
        
        # Process comparison result with the desired tolerance
        processed_result = process_output(comparison_result, tolerance=3)
        
        # Append processed result to the list
        processed_outputs.append(processed_result)
    
    # Analyze results for the current break type
    result_analysis = analyze_series_by_break_type(processed_outputs)
    
    # Print the analysis for this break type
    print(f"Analysis for series with {break_type} (3):")
    print(result_analysis)

Analysis for series with mean_shift (3):
({'mean_shift': {'true_positives': 722, 'false_positives': 28, 'true_negatives': 1120, 'false_negatives': 17, 'total_series': 1887, 'accuracy': 0.9761526232114467, 'precision': 0.9626666666666667, 'recall': 0.9769959404600812, 'f1_score': 0.9697783747481532}, 'variance_shift': {'true_positives': 0, 'false_positives': 35, 'true_negatives': 1852, 'false_negatives': 0, 'total_series': 1887, 'accuracy': 0.9814520402755696, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'trend_shift': {'true_positives': 0, 'false_positives': 48, 'true_negatives': 1839, 'false_negatives': 0, 'total_series': 1887, 'accuracy': 0.9745627980922098, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'anomaly': {'true_positives': 679, 'false_positives': 42, 'true_negatives': 1100, 'false_negatives': 66, 'total_series': 1887, 'accuracy': 0.9427662957074722, 'precision': 0.941747572815534, 'recall': 0.9114093959731544, 'f1_score': 0.9263301500682128}}, {'mean_shift': {'series_wit

In [461]:
for dataset, break_type in zip(
    [filtered_mean_shift],  # Only process mean shift series
    ['mean_shift']  # Corresponding break type
):
    # List to store processed outputs for the current break type
    processed_outputs = []

    # Process each series in the dataset
    for series in dataset:
        # Directly pass the entire series (dataframe) to compare_break_locations
        # This ensures both 'data' and relevant 'labels' are available
        comparison_result = compare_break_locations(model, series)
        
        # Process comparison result with the desired tolerance
        processed_result = process_output(comparison_result, tolerance=5)
        
        # Append processed result to the list
        processed_outputs.append(processed_result)
    
    # Analyze results for the current break type
    result_analysis = analyze_series_by_break_type(processed_outputs)
    
    # Print the analysis for this break type
    print(f"Analysis for series with {break_type} (5):")
    print(result_analysis)

Analysis for series with mean_shift (5):
({'mean_shift': {'true_positives': 721, 'false_positives': 29, 'true_negatives': 1120, 'false_negatives': 17, 'total_series': 1887, 'accuracy': 0.9756226815050344, 'precision': 0.9613333333333334, 'recall': 0.9769647696476965, 'f1_score': 0.9690860215053764}, 'variance_shift': {'true_positives': 0, 'false_positives': 35, 'true_negatives': 1852, 'false_negatives': 0, 'total_series': 1887, 'accuracy': 0.9814520402755696, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'trend_shift': {'true_positives': 0, 'false_positives': 48, 'true_negatives': 1839, 'false_negatives': 0, 'total_series': 1887, 'accuracy': 0.9745627980922098, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'anomaly': {'true_positives': 679, 'false_positives': 42, 'true_negatives': 1100, 'false_negatives': 66, 'total_series': 1887, 'accuracy': 0.9427662957074722, 'precision': 0.941747572815534, 'recall': 0.9114093959731544, 'f1_score': 0.9263301500682128}}, {'mean_shift': {'series_wit

In [462]:
for dataset, break_type in zip(
    [filtered_trend_shift],  # Only process mean shift series
    ['trend_shift']  # Corresponding break type
):
    # List to store processed outputs for the current break type
    processed_outputs = []

    # Process each series in the dataset
    for series in dataset:
        # Directly pass the entire series (dataframe) to compare_break_locations
        # This ensures both 'data' and relevant 'labels' are available
        comparison_result = compare_break_locations(model, series)
        
        # Process comparison result with the desired tolerance
        processed_result = process_output(comparison_result, tolerance=0)
        
        # Append processed result to the list
        processed_outputs.append(processed_result)
    
    # Analyze results for the current break type
    result_analysis = analyze_series_by_break_type(processed_outputs)
    
    # Print the analysis for this break type
    print(f"Analysis for short series with {break_type} (0):")
    print(result_analysis)

Analysis for short series with trend_shift (0):
({'mean_shift': {'true_positives': 0, 'false_positives': 28, 'true_negatives': 1977, 'false_negatives': 0, 'total_series': 2005, 'accuracy': 0.9860349127182045, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'variance_shift': {'true_positives': 0, 'false_positives': 16, 'true_negatives': 1989, 'false_negatives': 0, 'total_series': 2005, 'accuracy': 0.9920199501246882, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'trend_shift': {'true_positives': 673, 'false_positives': 44, 'true_negatives': 1097, 'false_negatives': 191, 'total_series': 2005, 'accuracy': 0.8827930174563591, 'precision': 0.9386331938633193, 'recall': 0.7789351851851852, 'f1_score': 0.8513598987982289}, 'anomaly': {'true_positives': 682, 'false_positives': 19, 'true_negatives': 1238, 'false_negatives': 66, 'total_series': 2005, 'accuracy': 0.9576059850374065, 'precision': 0.9728958630527818, 'recall': 0.9117647058823529, 'f1_score': 0.9413388543823327}}, {'mean_shift': {'s

In [463]:
for dataset, break_type in zip(
    [filtered_trend_shift],  # Only process mean shift series
    ['trend_shift']  # Corresponding break type
):
    # List to store processed outputs for the current break type
    processed_outputs = []

    # Process each series in the dataset
    for series in dataset:
        # Directly pass the entire series (dataframe) to compare_break_locations
        # This ensures both 'data' and relevant 'labels' are available
        comparison_result = compare_break_locations(model, series)
        
        # Process comparison result with the desired tolerance
        processed_result = process_output(comparison_result, tolerance=3)
        
        # Append processed result to the list
        processed_outputs.append(processed_result)
    
    # Analyze results for the current break type
    result_analysis = analyze_series_by_break_type(processed_outputs)
    
    # Print the analysis for this break type
    print(f"Analysis for short series with {break_type} (3):")
    print(result_analysis)

Analysis for short series with trend_shift (3):
({'mean_shift': {'true_positives': 0, 'false_positives': 28, 'true_negatives': 1977, 'false_negatives': 0, 'total_series': 2005, 'accuracy': 0.9860349127182045, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'variance_shift': {'true_positives': 0, 'false_positives': 16, 'true_negatives': 1989, 'false_negatives': 0, 'total_series': 2005, 'accuracy': 0.9920199501246882, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'trend_shift': {'true_positives': 662, 'false_positives': 55, 'true_negatives': 1097, 'false_negatives': 191, 'total_series': 2005, 'accuracy': 0.8773067331670823, 'precision': 0.9232914923291492, 'recall': 0.776084407971864, 'f1_score': 0.843312101910828}, 'anomaly': {'true_positives': 679, 'false_positives': 22, 'true_negatives': 1238, 'false_negatives': 66, 'total_series': 2005, 'accuracy': 0.9561097256857856, 'precision': 0.9686162624821684, 'recall': 0.9114093959731544, 'f1_score': 0.9391424619640387}}, {'mean_shift': {'ser

In [464]:
for dataset, break_type in zip(
    [filtered_trend_shift],  # Only process mean shift series
    ['trend_shift']  # Corresponding break type
):
    # List to store processed outputs for the current break type
    processed_outputs = []

    # Process each series in the dataset
    for series in dataset:
        # Directly pass the entire series (dataframe) to compare_break_locations
        # This ensures both 'data' and relevant 'labels' are available
        comparison_result = compare_break_locations(model, series)
        
        # Process comparison result with the desired tolerance
        processed_result = process_output(comparison_result, tolerance=5)
        
        # Append processed result to the list
        processed_outputs.append(processed_result)
    
    # Analyze results for the current break type
    result_analysis = analyze_series_by_break_type(processed_outputs)
    
    # Print the analysis for this break type
    print(f"Analysis for short series with {break_type} (5):")
    print(result_analysis)

Analysis for short series with trend_shift (5):
({'mean_shift': {'true_positives': 0, 'false_positives': 28, 'true_negatives': 1977, 'false_negatives': 0, 'total_series': 2005, 'accuracy': 0.9860349127182045, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'variance_shift': {'true_positives': 0, 'false_positives': 16, 'true_negatives': 1989, 'false_negatives': 0, 'total_series': 2005, 'accuracy': 0.9920199501246882, 'precision': 0.0, 'recall': 0, 'f1_score': 0}, 'trend_shift': {'true_positives': 654, 'false_positives': 63, 'true_negatives': 1097, 'false_negatives': 191, 'total_series': 2005, 'accuracy': 0.8733167082294264, 'precision': 0.9121338912133892, 'recall': 0.7739644970414201, 'f1_score': 0.8373879641485276}, 'anomaly': {'true_positives': 679, 'false_positives': 22, 'true_negatives': 1238, 'false_negatives': 66, 'total_series': 2005, 'accuracy': 0.9561097256857856, 'precision': 0.9686162624821684, 'recall': 0.9114093959731544, 'f1_score': 0.9391424619640387}}, {'mean_shift': {'s